# CiTO Citation Nanopublication Generator

This notebook generates typed citation nanopublications using the CiTO (Citation Typing Ontology) from a JSON configuration file.

## CiTO Citation Types
CiTO provides semantic annotation of citation intent, including:
- `cito:cites` - General citation
- `cito:usesDataFrom` - Uses data from cited work
- `cito:usesMethodIn` - Uses method from cited work
- `cito:extends` - Extends work from cited paper
- `cito:confirms` - Confirms findings of cited work
- `cito:obtainsBackgroundFrom` - Gets background information from cited work

In [1]:
import json
import sys
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd().parent))

from nanopub_utils import (
    NanopubGenerator, load_config, save_nanopub,
    make_uri, make_literal, validate_required_fields,
    PREFIXES
)

In [2]:
class CitoNanopubGenerator(NanopubGenerator):
    """Generator for CiTO citation nanopublications."""
    
    def __init__(self, config: dict, nanopub_config: dict):
        # Merge metadata with individual nanopub config
        merged_config = {
            **config.get('metadata', {}),
            **nanopub_config,
            'template_uri': config.get('template_uri'),
            'label': nanopub_config.get('nanopub_label', 'CiTO Citation')
        }
        super().__init__(merged_config)
        self.add_prefix('cito')
        self.add_prefix('fabio')
    
    def generate_assertion(self) -> str:
        """Generate the CiTO assertion graph."""
        citing_article = self.config['citing_article']
        citations = self.config.get('citations', [])
        
        # Normalize citing article URI
        if not citing_article.startswith('http'):
            citing_article = f"https://doi.org/{citing_article}"
        
        lines = [f"{self.sub_prefix}:assertion {{"]
        
        # Declare the citing article as a scholarly work
        lines.append(f"  <{citing_article}> a fabio:ScholarlyWork .")
        
        # Add each citation with its type
        for citation in citations:
            cited_article = citation['cited_article']
            citation_type = citation['citation_type']
            label = citation.get('label', '')
            
            # Normalize cited article URI
            if not cited_article.startswith('http'):
                cited_article = f"https://doi.org/{cited_article}"
            
            # Add the citation relationship
            lines.append(f"  <{citing_article}> {citation_type} <{cited_article}> .")
            
            # Add label for the cited work if provided
            if label:
                lines.append(f"  <{cited_article}> rdfs:label {make_literal(label)} .")
        
        lines.append("}")
        return "\n".join(lines)

In [3]:
# Configuration
CONFIG_FILE = "../config/vbae208/vbae208_cito.json"  # Change this to use different config
CONFIG_FILE = "../config/clenet2025/clenet2025_cito.json"  # Change this to use different config
OUTPUT_DIR = "../output/cito"

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [4]:
# Load configuration
config = load_config(CONFIG_FILE)

print(f"Source paper: {config['metadata']['source_paper']['title']}")
print(f"DOI: {config['metadata']['source_paper']['doi']}")
print(f"Number of CiTO nanopublications to generate: {len(config['nanopublications'])}")
print()

print("Available CiTO types:")
for cito_type, label in config.get('cito_types', {}).items():
    print(f"  {cito_type}: {label}")
print()

print("Citations to generate:")
for i, np_config in enumerate(config['nanopublications'], 1):
    for citation in np_config.get('citations', []):
        print(f"{i}. {citation['citation_type']} -> {citation.get('label', citation['cited_article'])}")

Source paper: QOMIC: quantum optimization for motif identification
DOI: 10.1093/bioadv/vbae208
Number of CiTO nanopublications to generate: 4

Available CiTO types:
  cito:cites: cites
  cito:citesAsDataSource: cites as data source
  cito:citesAsMetadataDocument: cites as metadata document
  cito:citesAsSourceDocument: cites as source document
  cito:usesDataFrom: uses data from
  cito:usesMethodIn: uses method in
  cito:extends: extends
  cito:obtainsBackgroundFrom: obtains background from
  cito:supports: supports
  cito:confirms: confirms
  cito:providesDataFor: provides data for
  cito:documents: documents

Citations to generate:
1. cito:usesDataFrom -> TRRUST v2 database paper
2. cito:usesMethodIn -> QAOA algorithm paper (Farhi et al. 2014)
3. cito:extends -> Network motifs foundational paper (Milo et al. 2002)
4. cito:usesMethodIn -> Qiskit quantum computing framework


In [5]:
# Generate nanopublications
generated_files = []

for np_config in config['nanopublications']:
    # Create generator
    generator = CitoNanopubGenerator(config, np_config)
    
    # Generate nanopub content
    nanopub_content = generator.generate()
    
    # Save to file
    output_file = f"{OUTPUT_DIR}/{np_config['id']}.trig"
    save_nanopub(nanopub_content, output_file)
    generated_files.append(output_file)
    
    print(f"Generated: {output_file}")

print(f"\nTotal generated: {len(generated_files)} nanopublications")

Generated: ../output/cito/cito_qomic_trrust.trig
Generated: ../output/cito/cito_qomic_qaoa.trig
Generated: ../output/cito/cito_qomic_motif.trig
Generated: ../output/cito/cito_qomic_qiskit.trig

Total generated: 4 nanopublications


In [6]:
# Preview first generated nanopublication
if generated_files:
    print(f"Preview of {generated_files[0]}:\n")
    print("=" * 80)
    with open(generated_files[0], 'r') as f:
        print(f.read())

Preview of ../output/cito/cito_qomic_trrust.trig:

@prefix this: <https://w3id.org/np/RAb222402036cbe8e9a34c05bf94db0f0a3869adb0498> .
@prefix sub: <https://w3id.org/np/RAb222402036cbe8e9a34c05bf94db0f0a3869adb0498/> .
@prefix cito: <http://purl.org/spar/cito/> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix fabio: <http://purl.org/spar/fabio/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix nt: <https://w3id.org/np/o/ntemplate/> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sub:Head {
  this: a np:Nanopublication ;
    np:hasAssertion sub:assertion ;
    np:hasProvenance sub:provenance ;
    np:hasPublicationInfo sub:pubinfo .
}

sub:assertion {
  <https://doi.org/10.1093/bioadv

## Next Steps

1. Review the generated `.trig` files in the output directory
2. Sign and publish using Nanodash or nanopub-java
3. To use with a different paper, create a new JSON config file and update `CONFIG_FILE`